In [37]:
import pandas as pd
from lifelines import CoxPHFitter
from src.helper_methods import *
from src.pipe_store import *
from src.constants import *

import warnings

warnings.filterwarnings('ignore', 'FutureWarning', )
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data Loading

In [17]:
path = '~/Repos/STRIDE/STRIDE-Analytics/data/20210614-mismatch_ep_db-extended.pickle'

donor_type ='Deceased'
# antibody_epitope = 
status = 'All'
num_col = ['DonorAge_NOTR', 'DialysisYears', 'RecipientAge_NOTR', 'CIPHour_DBD', 'CIPHour_DCD']

df = (
    data_loading(path)
    .pipe(start_pipeline, status, donor_type)
    .pipe(keeping_features,
        'Failure',
        'Survival[Y]',
        'EpvsHLA_Donor',
        '#DESA',
        'DESA', 
        'DonorAge_NOTR',
        'DialysisYears',
        'RecipientAge_NOTR',
        'CIPHour_DBD', 
        'CIPHour_DCD' ,
        'TypeCadaveric_NOTR', 
        'Donor_Type',
        'IL2rMoAb_T0'
    )
    .pipe(integer_encoder, 'TypeCadaveric_NOTR', 'IL2rMoAb_T0')
    .pipe(features_from_antibody_epitopes)
    .pipe(set_time_event_label, E='Failure', T='Survival[Y]')
    # .pipe(feature_scaler, num_col, scaler='standard')
    .pipe(censoring_deaths)
    .pipe(setting_prediction_horizon, 15)
    .rename(columns={'IL2rMoAb_T0':'IL2rMoAb'})
)
df = df.assign(
                No_DESA = df['DESA'].apply(lambda x: 0 if x else 1),
                DESA_12 = df['#DESA'].apply(lambda x: 1 if (x >= 1) & (x <= 2) else 0),
                DESA_3ormore = df['#DESA'].apply(lambda x: 1 if x >= 3 else 0),
        )

Step: data_loading | Shape: (4690, 24) | Computation Time: 0.021832s
- selceted cohort --> Donor Type: Deceased, Epitope Antibody Presence: All
Step: start_pipeline | Shape: (3235, 23) | Computation Time: 0.009697s
Step: integer_encoder | Shape: (3235, 13) | Computation Time: 0.00348s
Step: set_time_event_label | Shape: (3235, 17) | Computation Time: 0.003396s
Step: censoring_deaths | Shape: (3235, 17) | Computation Time: 0.00261s
Step: setting_prediction_horizon | Shape: (3235, 17) | Computation Time: 0.005386s


# Structural Causal Models

# Checking the Cox Proportional Hazard Assumptions

https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html?highlight=cox%20assumptions

The proportional hazard assumption is that all individuals have the same hazard function, but a unique scaling factor infront. So the shape of the hazard function is the same for all individuals, and only a scalar multiple changes per individual.

# Potential Outcomes

## Finding Inverse Probability Wieghts (IPW)

### Deceased Donor

In [43]:
from lifelines import CoxPHFitter

df_total = df.copy(deep=True)
df_late = df_total[df_total['T'] > 1]
df_early = df_total[df_total['T'] < 1]

# for df, title in [(df_total, 'Total Data Set'), (df_late, 'Late Failure'), (df_early, 'Early Failure')]:
confounders = [
    'RecipientAge_NOTR', 'RecipientAge_NOTR*RecipientAge_NOTR', 
    'DonorAge_NOTR', 'DonorAge_NOTR*DonorAge_NOTR', 'IL2rMoAb', 
    'CIPHour_DBD', 'CIPHour_DCD', 
]

df_treat_group = create_treatment_grups(df, [RELEVANT_DESA_BAD])
treatments = ['No_DESA', 'Other_DESA', 'Specific_DESA']
# treatments = ['No_DESA', 'Other_DESA', 'Specific_DESA', 'DESA_3more']
df_weight = find_ipw(df_treat_group, confounders, treatments, verbose=False)

cph = CoxPHFitter()
cols = [
    # 'No_DESA',
    'Specific_DESA',
    'Other_DESA',
    'E', 'T', 
    'w'
]
cph.fit(
    df_weight[cols], 
    weights_col='w',
    duration_col='T', event_col='E', robust=True)
print(cph.print_summary())

/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/utils/printer.py:62: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return summary_df[columns].to_latex(float_format="%." + str(self.decimals) + "f")


<lifelines.CoxPHFitter: fitted with 9688.73 total observations, 5829.95 right-censored observations>
             duration col = 'T'
                event col = 'E'
              weights col = 'w'
          robust variance = True
      baseline estimation = breslow
   number of observations = 9688.73
number of events observed = 3858.78
   partial log-likelihood = -33318.29
         time fit was run = 2022-04-06 14:10:02 UTC

---
                coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                             
Specific_DESA   1.17       3.21       0.12             0.93             1.40                 2.53                 4.06
Other_DESA      0.26       1.30       0.14            -0.01             0.53                 0.99                 1.71

                cmp to    z      p   -log2(p)
covariate                                    
Specific_DESA     0.00 9.67 <0.005      71.05
Other_DESA        0.00 1.91   0.06       4.17
---
Concordance = 0.54
Partial AIC = 66640.58
log-likelihood ratio test = 1003.58 on 2 df
-log2(p) of ll-ratio test = 723.93

None


In [39]:
cph.check_assumptions(df_weight[cols], p_value_threshold=0.05, show_plots=True)

Proportional hazard assumption looks okay.


/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/fitters/kaplan_meier_fitter.py:317: StatisticalWarning: It looks like your weights are not integers, possibly propensity scores then?
  It's important to know that the naive variance estimates of the coefficients are biased. Instead use Monte Carlo to
  estimate the variances. See paper "Variance estimation when using inverse probability of treatment weighting (IPTW) with survival analysis"
  or "Adjusted Kaplan-Meier estimator and log-rank test with inverse probability of treatment weighting for survival data."
                  
  warnings.warn(
/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/fitters/mixins.py:108: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  for variable in self.params_.index & (colu

[]

In [40]:
from lifelines.statistics import proportional_hazard_test
results = proportional_hazard_test(cph, df_weight[cols], time_transform='rank')
results.print_summary(decimals=3, model="untransformed variables")

/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/statistics.py:143: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return self.summary.to_latex()


<lifelines.StatisticalResult: proportional_hazard_test>
    time_transform = rank
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 9688.73 total observations, 5829.95 right-censored observations>
         test_name = proportional_hazard_test

---
               test_statistic    p  -log2(p)
Other_DESA               0.03 0.87      0.20
Specific_DESA            0.12 0.73      0.45

# Living Donor

In [52]:
path = '~/Repos/STRIDE/STRIDE-Analytics/data/20210614-mismatch_ep_db-extended.pickle'
donor_type ='Living'
status = 'All'
num_col = ['DonorAge_NOTR', 'DialysisYears', 'RecipientAge_NOTR', 'CIPHour_DBD', 'CIPHour_DCD']

df = (
    data_loading(path)
    .pipe(start_pipeline, status, donor_type)
    .pipe(keeping_features,
        'Failure',
        'Survival[Y]',
        'EpvsHLA_Donor',
        '#DESA',
        'DESA', 
        'DonorAge_NOTR',
        'DialysisYears',
        'RecipientAge_NOTR',
        'CIPHour_DBD', 
        'CIPHour_DCD' ,
        # 'TypeCadaveric_NOTR', 
        'Donor_Type',
        'IL2rMoAb_T0'
    )
    .pipe(integer_encoder, 'IL2rMoAb_T0')
    .pipe(features_from_antibody_epitopes)
    .pipe(set_time_event_label, E='Failure', T='Survival[Y]')
    # .pipe(feature_scaler, num_col, scaler='standard')
    .pipe(censoring_deaths)
    .pipe(setting_prediction_horizon, 15)
    .rename(columns={'IL2rMoAb_T0':'IL2rMoAb'})
)
df['DESA_3more'] = df['#DESA'].apply(lambda x: 1 if x > 2 else 0)

Step: data_loading | Shape: (4690, 24) | Computation Time: 0.01847s
- selceted cohort --> Donor Type: Living, Epitope Antibody Presence: All
Step: start_pipeline | Shape: (1455, 23) | Computation Time: 0.013016s
Step: integer_encoder | Shape: (1455, 12) | Computation Time: 0.000888s
Step: set_time_event_label | Shape: (1455, 16) | Computation Time: 0.004486s
Step: censoring_deaths | Shape: (1455, 16) | Computation Time: 0.002584s
Step: setting_prediction_horizon | Shape: (1455, 16) | Computation Time: 0.005771s


In [53]:
df.sample()

,#DESA,DESA,DonorAge_NOTR,DialysisYears,RecipientAge_NOTR,CIPHour_DBD,CIPHour_DCD,Donor_Type,IL2rMoAb,No_DESA,Relevant_DESA_Bad,Relevant_DESA_Good,Class_I,Class_II,E,T,DESA_3more
838,0,{},20,2.7,44,0.0,0.0,Living,0,1,0,0,0,0,0,15.0,0


In [16]:
df_total = df.copy(deep=True)
df_late = df_total[df_total['T'] > 1]
df_early = df_total[df_total['T'] < 1]

# for df, title in [(df_total, 'Total Data Set'), (df_late, 'Late Failure'), (df_early, 'Early Failure')]:
df = df_total
confounders = [
    'RecipientAge_NOTR', 'RecipientAge_NOTR*RecipientAge_NOTR', 
    'DonorAge_NOTR', 'DonorAge_NOTR*DonorAge_NOTR', 'IL2rMoAb', 
    'CIPHour_DBD', 'CIPHour_DCD', 
]

df_treat_group = create_treatment_grups(df, [RELEVANT_DESA_BAD])
# treatments = ['No_DESA', 'Other_DESA', 'Specific_DESA']
treatments = ['No_DESA', 'Other_DESA', 'Specific_DESA', 'DESA_3more']
# df_weight = find_ipw(df_treat_group, confounders, treatments, verbose=False)
df_treat_group
# cph = CoxPHFitter()
# cols = [
#     'DonorAge_NOTR', 'RecipientAge_NOTR',
#     'DialysisYears', 'IL2rMoAb',
#     'CIPHour_DBD', 'CIPHour_DCD', 
#     'No_DESA',
#     'Specific_DESA',
#     'DESA_3more',
#     'E', 'T', 'w',
# ]
# cph.fit(df_weight[cols], weights_col='w', duration_col='T', event_col='E', robust=True)
# # print(title)
# cph.print_summary()


,#DESA,DESA,DonorAge_NOTR,DialysisYears,RecipientAge_NOTR,CIPHour_DBD,CIPHour_DCD,TypeCadaveric_NOTR,Donor_Type,IL2rMoAb,...,Relevant_DESA_Bad,Relevant_DESA_Good,Class_I,Class_II,E,T,DESA_3more,Specific_DESA,Other_DESA,Groups
0,0,{},44,2.3,54,9.000000,0.000000,0,Deceased,0,...,0,0,0,0,0,12.191781,0,0,0,1
1,0,{},54,2.1,32,21.750000,0.000000,0,Deceased,0,...,0,0,0,0,0,15.000000,0,0,0,1
2,0,{},44,4.9,27,36.000000,0.000000,0,Deceased,0,...,0,0,0,0,1,6.956164,0,0,0,1
3,0,{},23,0.0,40,30.000000,0.000000,0,Deceased,1,...,0,0,0,0,0,4.509589,0,0,0,1
4,0,{},55,5.4,38,0.000000,17.216667,1,Deceased,0,...,0,0,0,0,1,0.010959,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4685,0,{},51,1.5,43,29.000000,0.000000,0,Deceased,1,...,0,0,0,0,0,12.126027,0,0,0,1
4686,0,{},54,1.7,37,20.700000,0.000000,0,Deceased,1,...,0,0,0,0,1,10.879452,0,0,0,1
4687,0,{},21,3.5,46,17.183333,0.000000,0,Deceased,1,...,0,0,0,0,0,15.000000,0,0,0,1
4688,0,{},42,2.5,23,0.000000,32.183333,1,Deceased,1,...,0,0,0,0,1,11.816438,0,0,0,1


# Relevant DESA Good

In [17]:
df_new = df.copy(deep=True)

confounders = [
    'RecipientAge_NOTR', 'RecipientAge_NOTR*RecipientAge_NOTR', 
    'DonorAge_NOTR', 'DonorAge_NOTR*DonorAge_NOTR', 'IL2rMoAb', 
    'CIPHour_DBD', 'CIPHour_DCD', 
]

df_treat_group = create_treatment_grups(df_new, [RELEVANT_DESA_GOOD])
treatments = ['No_DESA', 'Other_DESA', 'Specific_DESA']
num_col = ['DonorAge_NOTR', 'DialysisYears', 'RecipientAge_NOTR', 'CIPHour_DBD', 'CIPHour_DCD']
df_weight = find_ipw(df_treat_group, confounders, treatments,  verbose=False)

cph = CoxPHFitter()
cols = [
    'DonorAge_NOTR', 'RecipientAge_NOTR',
    'DialysisYears', 'IL2rMoAb',
    'CIPHour_DBD', 'CIPHour_DCD', 
    'No_DESA',
    'Specific_DESA',
    'E', 'T', 'w',
]
cph.fit(df_weight[cols], weights_col='w', duration_col='T', event_col='E', robust=True)
cph.print_summary()

Daat Columns Index(['#DESA', 'DESA', 'DonorAge_NOTR', 'DialysisYears', 'RecipientAge_NOTR',
       'CIPHour_DBD', 'CIPHour_DCD', 'TypeCadaveric_NOTR', 'Donor_Type',
       'IL2rMoAb', 'No_DESA', 'Relevant_DESA_Bad', 'Relevant_DESA_Good',
       'Class_I', 'Class_II', 'E', 'T', 'DESA_3more', 'Specific_DESA',
       'Other_DESA', 'Groups'],
      dtype='object')


/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/utils/printer.py:62: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return summary_df[columns].to_latex(float_format="%." + str(self.decimals) + "f")


<lifelines.CoxPHFitter: fitted with 9688.9 total observations, 6110.86 right-censored observations>
             duration col = 'T'
                event col = 'E'
              weights col = 'w'
          robust variance = True
      baseline estimation = breslow
   number of observations = 9688.9
number of events observed = 3578.04
   partial log-likelihood = -30719.27
         time fit was run = 2022-04-05 12:49:44 UTC

---
                    coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                 
DonorAge_NOTR       0.26       1.30       0.08             0.11             0.42                 1.12                 1.52
RecipientAge_NOTR  -0.22       0.80       0.07            -0.35            -0.08                 0.70                 0.92
DialysisYears       0.06       1.06       0.05            -0.04             0.16                 0.96                 1.18
IL2rMoAb           -0.11       0.90       0.17            -0.44             0.22                 0.65                 1.25
CIPHour_DBD         0.20       1.22       0.11            -0.01             0.41                 0.99                 1.50
CIPHour_DCD         0.38       1.47       0.10             0.19             0.58                 1.21                 1.79
No_DESA            -0.97       0.38       0.11            -1.18            -0.76                 0.31                 0.47
Specific_DESA      -0.78       0.46       0.19            -1.15            -0.41                 0.32                 0.67

                    cmp to     z      p   -log2(p)
covariate                                         
DonorAge_NOTR         0.00  3.38 <0.005      10.43
RecipientAge_NOTR     0.00 -3.17 <0.005       9.36
DialysisYears         0.00  1.20   0.23       2.12
IL2rMoAb              0.00 -0.63   0.53       0.92
CIPHour_DBD           0.00  1.84   0.07       3.93
CIPHour_DCD           0.00  3.84 <0.005      12.97
No_DESA               0.00 -9.13 <0.005      63.65
Specific_DESA         0.00 -4.10 <0.005      14.54
---
Concordance = 0.64
Partial AIC = 61454.53
log-likelihood ratio test = 1291.61 on 8 df
-log2(p) of ll-ratio test = 906.27

In [8]:
cph.check_assumptions(df_new, p_value_threshold=0.05, show_plots=True)

Proportional hazard assumption looks okay.


/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/fitters/kaplan_meier_fitter.py:317: StatisticalWarning: It looks like your weights are not integers, possibly propensity scores then?
  It's important to know that the naive variance estimates of the coefficients are biased. Instead use Monte Carlo to
  estimate the variances. See paper "Variance estimation when using inverse probability of treatment weighting (IPTW) with survival analysis"
  or "Adjusted Kaplan-Meier estimator and log-rank test with inverse probability of treatment weighting for survival data."
                  
  warnings.warn(
/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/fitters/mixins.py:108: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  for variable in self.params_.index & (colu

[]

In [9]:
from lifelines.statistics import proportional_hazard_test

results = proportional_hazard_test(cph, df_new, time_transform='rank')
results.print_summary(decimals=3, model="untransformed variables")

/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/statistics.py:143: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return self.summary.to_latex()


<lifelines.StatisticalResult: proportional_hazard_test>
    time_transform = rank
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 9688.9 total observations, 6110.86 right-censored observations>
         test_name = proportional_hazard_test

---
                   test_statistic    p  -log2(p)
CIPHour_DBD                  0.09 0.76      0.39
CIPHour_DCD                  0.11 0.74      0.43
Class_I                      0.01 0.94      0.08
Class_II                     0.00 0.97      0.04
DialysisYears                0.03 0.87      0.20
DonorAge_NOTR                0.00 0.97      0.04
IL2rMoAb                     0.00 0.97      0.05
No_DESA                      0.02 0.89      0.16
RecipientAge_NOTR            0.27 0.60      0.73
Specific_DESA                0.00 0.96      0.06

# Table Hazard Ratios

## Deceased Donors

In [20]:
path = '~/Repos/STRIDE/STRIDE-Analytics/data/20210614-mismatch_ep_db-extended.pickle'
donor_type ='Deceased'
status = 'All'
antibody_epitope = True
num_col = ['DonorAge_NOTR', 'DialysisYears', 'RecipientAge_NOTR', 'CIPHour_DBD', 'CIPHour_DCD']

df_deceased = (
    data_loading(path)
    .pipe(start_pipeline, status, donor_type)
    .pipe(keeping_features,
        'Failure',
        'Survival[Y]',
        'EpvsHLA_Donor',
        '#DESA',
        'DESA', 
        'DonorAge_NOTR',
        'DialysisYears',
        'RecipientAge_NOTR',
        'CIPHour_DBD', 
        'CIPHour_DCD' ,
        'Donor_Type',
        'IL2rMoAb_T0'
    )
    .pipe(integer_encoder, 'IL2rMoAb_T0')
    # .pipe(polynomial_power2, 'DonorAge_NOTR', 'RecipientAge_NOTR') 
    .pipe(eng_immunological_features, antibody_epitope=antibody_epitope)
    .pipe(set_time_event_label, E='Failure', T='Survival[Y]')
    .pipe(censoring_deaths)
    # .pipe(feature_scaler, num_col, scaler='standard')
    .rename(columns={'IL2rMoAb_T0':'IL2rMoAb'})
    # .pipe(setting_prediction_horizon, 10)
)
# df_deceased['No_DESA'] = df_deceased['#DESA'].apply(lambda x: 1 if not x else 0)
# df_deceased['Early_Failures'] = df_deceased[df_deceased['T'] <= 1]['No_DESA'].apply(lambda x: 1 if x == 0 else 0 )
df_deceased['DESA_12'] = df_deceased['#DESA'].apply(lambda x: 1 if (x >= 1) & (x <= 2) else 0)
df_deceased['DESA_3more'] = df_deceased['#DESA'].apply(lambda x: 1 if x > 2 else 0)
# df_deceased

Step: data_loading | Shape: (4690, 24) | Computation Time: 0.01404s
- selceted cohort --> Donor Type: Deceased, Epitope Antibody Presence: All
Step: start_pipeline | Shape: (3235, 23) | Computation Time: 0.00686s
Step: integer_encoder | Shape: (3235, 12) | Computation Time: 0.001241s
Step: eng_immunological_features | Shape: (3235, 16) | Computation Time: 0.015234s
Step: set_time_event_label | Shape: (3235, 16) | Computation Time: 0.00277s
Step: censoring_deaths | Shape: (3235, 16) | Computation Time: 0.001054s


In [73]:
confounders = [
    'RecipientAge_NOTR', 
    'RecipientAge_NOTR*RecipientAge_NOTR', 
    'DonorAge_NOTR', 
    'DonorAge_NOTR*DonorAge_NOTR', 
    'IL2rMoAb', 
    # 'CIPHour_DBD', 'CIPHour_DCD',
]
num_col = ['DonorAge_NOTR', 'RecipientAge_NOTR',
            # 'DonorAge_NOTR^2', 'RecipientAge_NOTR^2',
            'DialysisYears', 'CIPHour_DBD', 'CIPHour_DCD']

df_deceased = create_treatment_grups(df_deceased, [RELEVANT_DESA_BAD])
treatments = ['No_DESA', 'Other_DESA', 'Specific_DESA']
df_weight = find_ipw(df_deceased, confounders, treatments, scaler='standard', num_col=num_col)
# df_weight = df_deceased
df_weight = df_weight[[
    # 'DonorAge_NOTR', 'RecipientAge_NOTR',
    # 'DonorAge_NOTR^2', 'RecipientAge_NOTR^2',
    # 'CIPHour_DBD', 'CIPHour_DCD', 
    # 'IL2rMoAb', 
    'No_DESA', 
    # 'Specific_DESA', 
    'Relevant_DESA_Bad',
    # 'DialysisYears',
    # 'Early_Failures', 
    # 'DESA 1-2',
    # 'DESA_3more',
    'T', 'E', 
    'w',
]]
cph = CoxPHFitter()
cph.fit(df_weight, 
        weights_col='w', 
        duration_col='T', event_col='E', robust=True)
cph.print_summary()

NameError: name 'df_deceased' is not defined

In [34]:
confounders = [
    'RecipientAge_NOTR', 
    'RecipientAge_NOTR*RecipientAge_NOTR', 
    'DonorAge_NOTR', 
    'DonorAge_NOTR*DonorAge_NOTR', 
    'IL2rMoAb', 
    # 'CIPHour_DBD', 'CIPHour_DCD',
]
num_col = ['DonorAge_NOTR', 'RecipientAge_NOTR',
            # 'DonorAge_NOTR^2', 'RecipientAge_NOTR^2',
            'DialysisYears', 'CIPHour_DBD', 'CIPHour_DCD']
df_deceased_early = df_deceased[df_deceased['T'] < 1]
df_deceased_early = create_treatment_grups(df_deceased_early, [RELEVANT_DESA_BAD])
treatments = ['No_DESA', 'Other_DESA', 'Specific_DESA']
df_weight = find_ipw(df_deceased_early, confounders, treatments, scaler='standard', num_col=num_col)
# df_weight = df_deceased
df_weight = df_weight[[
    'DonorAge_NOTR', 'RecipientAge_NOTR',
    # 'DonorAge_NOTR^2', 'RecipientAge_NOTR^2',
    'CIPHour_DBD', 'CIPHour_DCD', 
    'IL2rMoAb', 
    'No_DESA', 
    # 'Specific_DESA', 
    'Relevant_DESA_Bad',
    'DialysisYears',
    # 'Early_Failures', 
    # 'DESA 1-2',
    # 'DESA_3more',
    'T', 'E', 
    'w',
]]
cph = CoxPHFitter()
cph.fit(df_weight, 
        weights_col='w', 
        duration_col='T', event_col='E', robust=True)
cph.print_summary()

/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/utils/printer.py:62: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return summary_df[columns].to_latex(float_format="%." + str(self.decimals) + "f")


<lifelines.CoxPHFitter: fitted with 1208.77 total observations, 297.669 right-censored observations>
             duration col = 'T'
                event col = 'E'
              weights col = 'w'
          robust variance = True
      baseline estimation = breslow
   number of observations = 1208.77
number of events observed = 911.096
   partial log-likelihood = -5626.23
         time fit was run = 2022-03-24 15:30:50 UTC

---
                    coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                 
DonorAge_NOTR       0.05       1.06       0.10            -0.14             0.25                 0.87                 1.28
RecipientAge_NOTR  -0.15       0.86       0.09            -0.32             0.03                 0.73                 1.03
CIPHour_DBD        -0.28       0.75       0.18            -0.63             0.06                 0.53                 1.06
CIPHour_DCD        -0.06       0.94       0.16            -0.39             0.26                 0.68                 1.30
IL2rMoAb            0.29       1.34       0.24            -0.18             0.76                 0.84                 2.14
No_DESA             0.05       1.05       0.27            -0.48             0.57                 0.62                 1.77
Relevant_DESA_Bad   0.62       1.87       0.32             0.01             1.24                 1.01                 3.47
DialysisYears      -0.11       0.90       0.09            -0.28             0.06                 0.76                 1.06

                    cmp to     z    p   -log2(p)
covariate                                       
DonorAge_NOTR         0.00  0.54 0.59       0.76
RecipientAge_NOTR     0.00 -1.63 0.10       3.29
CIPHour_DBD           0.00 -1.60 0.11       3.20
CIPHour_DCD           0.00 -0.38 0.70       0.51
IL2rMoAb              0.00  1.21 0.22       2.15
No_DESA               0.00  0.18 0.86       0.22
Relevant_DESA_Bad     0.00  1.98 0.05       4.38
DialysisYears         0.00 -1.28 0.20       2.33
---
Concordance = 0.60
Partial AIC = 11268.46
log-likelihood ratio test = 124.07 on 8 df
-log2(p) of ll-ratio test = 74.15

In [35]:
confounders = [
    'RecipientAge_NOTR', 
    'RecipientAge_NOTR*RecipientAge_NOTR', 
    'DonorAge_NOTR', 
    'DonorAge_NOTR*DonorAge_NOTR', 
    'IL2rMoAb', 
    # 'CIPHour_DBD', 'CIPHour_DCD',
]
num_col = ['DonorAge_NOTR', 'RecipientAge_NOTR',
            # 'DonorAge_NOTR^2', 'RecipientAge_NOTR^2',
            'DialysisYears', 'CIPHour_DBD', 'CIPHour_DCD']
df_deceased_late = df_deceased[df_deceased['T'] > 1]
df_deceased_early = create_treatment_grups(df_deceased_early, [RELEVANT_DESA_BAD])
treatments = ['No_DESA', 'Other_DESA', 'Specific_DESA']
df_weight = find_ipw(df_deceased_early, confounders, treatments, scaler='standard', num_col=num_col)
# df_weight = df_deceased
df_weight = df_weight[[
    'DonorAge_NOTR', 'RecipientAge_NOTR',
    # 'DonorAge_NOTR^2', 'RecipientAge_NOTR^2',
    'CIPHour_DBD', 'CIPHour_DCD', 
    'IL2rMoAb', 
    'No_DESA', 
    # 'Specific_DESA', 
    'Relevant_DESA_Bad',
    'DialysisYears',
    # 'Early_Failures', 
    # 'DESA 1-2',
    # 'DESA_3more',
    'T', 'E', 
    'w',
]]
cph = CoxPHFitter()
cph.fit(df_weight, 
        weights_col='w', 
        duration_col='T', event_col='E', robust=True)
cph.print_summary()

/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/utils/printer.py:62: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return summary_df[columns].to_latex(float_format="%." + str(self.decimals) + "f")


<lifelines.CoxPHFitter: fitted with 1208.77 total observations, 297.669 right-censored observations>
             duration col = 'T'
                event col = 'E'
              weights col = 'w'
          robust variance = True
      baseline estimation = breslow
   number of observations = 1208.77
number of events observed = 911.096
   partial log-likelihood = -5626.23
         time fit was run = 2022-03-24 15:31:27 UTC

---
                    coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                 
DonorAge_NOTR       0.05       1.06       0.10            -0.14             0.25                 0.87                 1.28
RecipientAge_NOTR  -0.15       0.86       0.09            -0.32             0.03                 0.73                 1.03
CIPHour_DBD        -0.28       0.75       0.18            -0.63             0.06                 0.53                 1.06
CIPHour_DCD        -0.06       0.94       0.16            -0.39             0.26                 0.68                 1.30
IL2rMoAb            0.29       1.34       0.24            -0.18             0.76                 0.84                 2.14
No_DESA             0.05       1.05       0.27            -0.48             0.57                 0.62                 1.77
Relevant_DESA_Bad   0.62       1.87       0.32             0.01             1.24                 1.01                 3.47
DialysisYears      -0.11       0.90       0.09            -0.28             0.06                 0.76                 1.06

                    cmp to     z    p   -log2(p)
covariate                                       
DonorAge_NOTR         0.00  0.54 0.59       0.76
RecipientAge_NOTR     0.00 -1.63 0.10       3.29
CIPHour_DBD           0.00 -1.60 0.11       3.20
CIPHour_DCD           0.00 -0.38 0.70       0.51
IL2rMoAb              0.00  1.21 0.22       2.15
No_DESA               0.00  0.18 0.86       0.22
Relevant_DESA_Bad     0.00  1.98 0.05       4.38
DialysisYears         0.00 -1.28 0.20       2.33
---
Concordance = 0.60
Partial AIC = 11268.46
log-likelihood ratio test = 124.07 on 8 df
-log2(p) of ll-ratio test = 74.15